In [38]:
import numpy as np
import pandas as pd
import json 

# Loading data


In [39]:
data = json.load(open("Client_data.json"))

In [40]:
complaint = []

In [42]:
data

[{'_id': '3211475',
  '_index': 'complaint-public-v2',
  '_score': 0.0,
  '_source': {'company': 'JPMORGAN CHASE & CO.',
   'company_public_response': None,
   'company_response': 'Closed with explanation',
   'complaint_id': '3211475',
   'complaint_what_happened': '',
   'consumer_consent_provided': 'Consent not provided',
   'consumer_disputed': 'N/A',
   'date_received': '2019-04-13T12:00:00-05:00',
   'date_sent_to_company': '2019-04-13T12:00:00-05:00',
   'issue': 'Attempts to collect debt not owed',
   'product': 'Debt collection',
   'state': 'CA',
   'sub_issue': 'Debt is not yours',
   'sub_product': 'Credit card debt',
   'submitted_via': 'Web',
   'tags': None,
   'timely': 'Yes',
   'zip_code': '90301'},
  '_type': 'complaint'},
 {'_id': '3229299',
  '_index': 'complaint-public-v2',
  '_score': 0.0,
  '_source': {'company': 'JPMORGAN CHASE & CO.',
   'company_public_response': None,
   'company_response': 'Closed with explanation',
   'complaint_id': '3229299',
   'complai

In [43]:
data[1]["_source"]["complaint_what_happened"]

'Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase Bank cardmember services. \nIn 2018 I wrote to Chase asking for debt verification and what they sent me a statement which is not acceptable. I am asking the bank to validate the debt. Instead I been receiving mail every month from them attempting to collect a debt. \nI have a right to know this information as a consumer. \n\nChase account # XXXX XXXX XXXX XXXX Thanks in advance for your help.'

In [44]:
for x in data:
    complaint.append(str(x["_source"]["complaint_what_happened"])+" " +str(x["_source"]["product"]) +" "+ str(x["_source"]["sub_product"]))

In [45]:
len(complaint)

78313

In [46]:
complaint = np.array(complaint)

In [47]:
complaint = complaint[complaint != np.array(None)]

In [48]:
import nltk
import re

In [49]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation

# Preproceessing data

In [50]:
def preprocessing(texts):
   
    texts = [re.findall(r'\w+', line.lower()) for line in texts]
    texts = [remove_stopwords(' '.join(line)).split() for line in texts]
    texts = [strip_punctuation(' '.join(line)).split() for line in texts]
    texts = [[token for token in line if len(token) >2] for line in texts]
    texts = [[token for token in line if not token.isnumeric()] for line in texts]
    lemmatizer = WordNetLemmatizer()
    texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]

    return texts

In [51]:
pre_processed_complaints = preprocessing(complaint)

In [52]:
pre_processed_complaints[1]

['good',
 'morning',
 'xxxx',
 'xxxx',
 'appreciate',
 'help',
 'stop',
 'chase',
 'bank',
 'cardmember',
 'services',
 'wrote',
 'chase',
 'asking',
 'debt',
 'verification',
 'sent',
 'statement',
 'acceptable',
 'asking',
 'bank',
 'validate',
 'debt',
 'instead',
 'receiving',
 'mail',
 'month',
 'attempting',
 'collect',
 'debt',
 'right',
 'know',
 'information',
 'consumer',
 'chase',
 'account',
 'xxxx',
 'xxxx',
 'xxxx',
 'xxxx',
 'thanks',
 'advance',
 'help',
 'debt',
 'collection',
 'credit',
 'card',
 'debt']

In [53]:
pre_processed_text = list(filter(None,pre_processed_complaints))

In [54]:
pre_processed_text

[['debt', 'collection', 'credit', 'card', 'debt'],
 ['good',
  'morning',
  'xxxx',
  'xxxx',
  'appreciate',
  'help',
  'stop',
  'chase',
  'bank',
  'cardmember',
  'services',
  'wrote',
  'chase',
  'asking',
  'debt',
  'verification',
  'sent',
  'statement',
  'acceptable',
  'asking',
  'bank',
  'validate',
  'debt',
  'instead',
  'receiving',
  'mail',
  'month',
  'attempting',
  'collect',
  'debt',
  'right',
  'know',
  'information',
  'consumer',
  'chase',
  'account',
  'xxxx',
  'xxxx',
  'xxxx',
  'xxxx',
  'thanks',
  'advance',
  'help',
  'debt',
  'collection',
  'credit',
  'card',
  'debt'],
 ['upgraded',
  'xxxx',
  'xxxx',
  'card',
  'told',
  'agent',
  'upgrade',
  'anniversary',
  'date',
  'change',
  'turned',
  'agent',
  'giving',
  'wrong',
  'information',
  'order',
  'upgrade',
  'account',
  'xxxx',
  'changed',
  'anniversary',
  'date',
  'xxxx',
  'xxxx',
  'consent',
  'xxxx',
  'recording',
  'agent',
  'misled',
  'credit',
  'card',
  

In [55]:
from gensim.corpora import Dictionary

In [56]:
dict_data = Dictionary(pre_processed_text)

In [57]:
corpus = [dict_data.doc2bow(text) for text in pre_processed_text]

In [58]:
from gensim.models import LdaModel
from sklearn.decomposition import NMF

# Model traning

In [79]:
model = LdaModel(corpus=corpus,id2word=dict_data,num_topics=5,chunksize=600,passes=30)

In [110]:
model.save("model_final")

C:\Users\naaz_baig\AppData\Local\Continuum\anaconda3\envs\py356\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [111]:
model = LdaModel.load("model_final")

C:\Users\naaz_baig\AppData\Local\Continuum\anaconda3\envs\py356\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [82]:
model.print_topics(num_topics=5,num_words=3)

[(0, '0.313*"card" + 0.219*"credit" + 0.056*"charge"'),
 (1, '0.158*"credit" + 0.062*"reporting" + 0.056*"debt"'),
 (2, '0.270*"account" + 0.117*"checking" + 0.116*"bank"'),
 (3, '0.322*"mortgage" + 0.068*"loan" + 0.057*"conventional"'),
 (4, '0.126*"xxxx" + 0.046*"chase" + 0.009*"told"')]

1 - Banking Services
0 - Loans
2 - Fraudulent Reporting
3 - Card
4 - others

In [83]:
model.show_topics(num_topics=5)

[(0,
  '0.313*"card" + 0.219*"credit" + 0.056*"charge" + 0.037*"general" + 0.037*"prepaid" + 0.036*"purpose" + 0.011*"cards" + 0.008*"balance" + 0.007*"points" + 0.006*"fee"'),
 (1,
  '0.158*"credit" + 0.062*"reporting" + 0.056*"debt" + 0.043*"consumer" + 0.041*"loan" + 0.032*"personal" + 0.031*"report" + 0.031*"collection" + 0.030*"reports" + 0.026*"services"'),
 (2,
  '0.270*"account" + 0.117*"checking" + 0.116*"bank" + 0.076*"service" + 0.045*"savings" + 0.035*"money" + 0.026*"check" + 0.017*"funds" + 0.016*"product" + 0.013*"transfer"'),
 (3,
  '0.322*"mortgage" + 0.068*"loan" + 0.057*"conventional" + 0.045*"home" + 0.035*"fixed" + 0.016*"modification" + 0.015*"student" + 0.015*"property" + 0.014*"line" + 0.014*"equity"'),
 (4,
  '0.126*"xxxx" + 0.046*"chase" + 0.009*"told" + 0.008*"payment" + 0.008*"called" + 0.007*"received" + 0.007*"time" + 0.007*"said" + 0.005*"bank" + 0.005*"letter"')]

In [84]:
model.get_document_topics(corpus[2])

[(0, 0.39963576), (2, 0.016381752), (4, 0.573912)]

In [85]:
model.log_perplexity(corpus)

-6.36663831502579

In [86]:
!pip install pyldavis

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [87]:
import pyLDAvis
import pyLDAvis.gensim as ldavis
pyLDAvis.enable_notebook()
vis = ldavis.prepare(model, corpus, dict_data)
vis

C:\Users\naaz_baig\AppData\Local\Continuum\anaconda3\envs\py356\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      67.877808        1       1  0.109582 -0.055911
2      10.226349        1       2  0.354228 -0.117513
0       9.653274        1       3 -0.254568 -0.168561
1       6.284384        1       4 -0.190678 -0.065022
3       5.958192        1       5 -0.018563  0.407008, topic_info=      Category           Freq          Term          Total  loglift  logprob
term                                                                        
2713   Default   79949.000000          card   79949.000000  30.0000  30.0000
4035   Default   80781.000000       account   80781.000000  29.0000  29.0000
11646  Default   82288.000000        credit   82288.000000  28.0000  28.0000
23675  Default   50820.000000      mortgage   50820.000000  27.0000  27.0000
25656  Default  226340.000000          xxxx  226340.000000  26.0000  26.0000
22934  Default   41260.000000          bank   41260.000000  25.0000  25.0000
25923  Default   31667.000000      checking   31667.000000  24.0000  24.0000
16156  Default   21820.000000       service   21820.000000  23.0000  23.0000
17911  Default   17455.000000          loan   17455.000000  22.0000  22.0000
26795  Default   15846.000000        charge   15846.000000  21.0000  21.0000
19071  Default   83333.000000         chase   83333.000000  20.0000  20.0000
4046   Default   10355.000000     reporting   10355.000000  19.0000  19.0000
11582  Default   12268.000000       savings   12268.000000  18.0000  18.0000
4183   Default   14600.000000         money   14600.000000  17.0000  17.0000
21263  Default    9285.000000          debt    9285.000000  16.0000  16.0000
17955  Default    9076.000000  conventional    9076.000000  15.0000  15.0000
23631  Default    8561.000000          home    8561.000000  14.0000  14.0000
11816  Default    9520.000000       general    9520.000000  13.0000  13.0000
24490  Default    9499.000000       prepaid    9499.000000  12.0000  12.0000
11152  Default    9330.000000       purpose    9330.000000  11.0000  11.0000
1653   Default    7783.000000      consumer    7783.000000  10.0000  10.0000
26824  Default   10824.000000         check   10824.000000   9.0000   9.0000
12471  Default    8025.000000        report    8025.000000   8.0000   8.0000
19230  Default    5592.000000         fixed    5592.000000   7.0000   7.0000
22289  Default    6318.000000      personal    6318.000000   6.0000   6.0000
25735  Default    5148.000000    collection    5148.000000   5.0000   5.0000
17263  Default    5037.000000       reports    5037.000000   4.0000   4.0000
9899   Default    5425.000000      services    5425.000000   3.0000   3.0000
413    Default    6895.000000         funds    6895.000000   2.0000   2.0000
442    Default    9569.000000   information    9569.000000   1.0000   1.0000
...        ...            ...           ...            ...      ...      ...
18148   Topic5     707.573975     refinance     708.459839   2.8192  -5.4076
6678    Topic5     505.547058     principal     506.438812   2.8186  -5.7438
15026   Topic5     494.440033     appraisal     495.323883   2.8186  -5.7660
17858   Topic5     367.512054      servicer     368.392578   2.8180  -6.0627
14906   Topic5     339.890930           pmi     340.767090   2.8178  -6.1408
18330   Topic5     318.550018      investor     319.436798   2.8176  -6.2056
20677   Topic5     295.407532     predatory     296.408295   2.8170  -6.2811
2639    Topic5     254.109833     mortgages     255.016861   2.8168  -6.4317
16139   Topic5     237.738708      borrower     238.620773   2.8167  -6.4982
22854   Topic5     236.797668     foreclose     237.676865   2.8167  -6.5022
4450    Topic5     236.265427        modify     237.154007   2.8166  -6.5045
21765   Topic5     226.455963   forbearance     227.339935   2.8165  -6.5469
14577   Topic5     222.426682       housing     223.329559   2.8164  -6.5648
22667   Topic5     213.752457     red

In [99]:
def result(x):
    max_tup = (0,0)
    for i in x:
        if i[1]>max_tup[1]:
            max_tup = i
    return max_tup[0]

In [100]:
model.get_document_topics(corpus[0])

[(0, 0.24925222),
 (1, 0.6507478),
 (2, 0.033333335),
 (3, 0.033333335),
 (4, 0.033333335)]

In [101]:
complaint[1]

'Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase Bank cardmember services. \nIn 2018 I wrote to Chase asking for debt verification and what they sent me a statement which is not acceptable. I am asking the bank to validate the debt. Instead I been receiving mail every month from them attempting to collect a debt. \nI have a right to know this information as a consumer. \n\nChase account # XXXX XXXX XXXX XXXX Thanks in advance for your help. Debt collection Credit card debt'

In [102]:
print(complaint)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [103]:
complaint = list(filter(None,complaint))

In [104]:
topics = []
for i in corpus:
    topics.append(result(model.get_document_topics(i)) + 1)

2 - Banking
4 - Loan
5 - Fraud
3 - Card
1 - Others

In [105]:
df_results= pd.DataFrame([complaint,topics]).T

In [106]:
df_results.columns = ["complaint","type"]

In [107]:
pd.set_option('display.max_colwidth', 500)

In [108]:
df_results.sample(40)

,complaint,type
5849,Debt collection Credit card debt,2
58559,Mortgage Other mortgage,4
48842,Chase is not doing what they said they would for under water and people behind on their mortage they did nothing with the arrears did n't not lower my mortgage and they are now foreclosing on my wife and I which we are XXXX. And I have been trying to get a modification and they go up on monthly payments Mortgage Conventional adjustable mortgage (ARM),4
22732,"Today I went to chase bank in XXXX XXXX at XXXX to open a bank account. I was told to have a seat and someone would be right with me. After waiting about 25 minutes I wanted to ask if someone was available to help me, so I approach a young blonde female and as I approach her window she walked away to talk to a young XXXX XXXX female. As they were smiling and trying to avoid me I felt so upset I walked away but as I was walking away I noticed the blonde female walking back up to the window so...",5
53582,Bank account or service Checking account,3
75120,"The reason for my dispute is with 6 credits cards companies is to try to get my money back for using them to buy XXXX memberships and start an on line business but XXXX has been shut down for an investigation by the FTC for fraudulent activity and being a scam. \n-Description of Services/Products : XXXX is a business education platform. I paid and registered for a training program, sponsored by XXXX XXXX, to learn about online business practices and marketing techniques. XXXX XXXX which we...",5
29976,"I 'm former account holder with Chase, I say former because over the past 3 years all my accounts at Chase ( checking / savings / credit card ) have been jacked four times - very interesting that only my accounts with Chase have been affected, none of my other accounts with other financial institution have been ever affected. \nAnyhow, On XXXX XXXX my Chase Freedom Account was part of a fraudulent scheme yet again ( over XXXX funds transfer / point redemption, etc. ). I went to the nearest b...",5
178,"When booking a hotel with XXXX, I checked the properties listed ( 4 ) as the stay being in one of the four. There was not a listing for "" XXXX ''. XXXX, XXXX, XXXX and one other was shown. Again -- no "" XXXX '' listing. I stayed at the XXXX in 2014 and it was TERRIBLE. Feeling safe, I hit "" book it now '', and what turns out to be the property? The XXXX. XXXX hid the name "" XXXX '' by using the name XXXX. I immediately called XXXX to rebook to another property. XXXX service center ran me a...",5
5050,Credit card or prepaid card General-purpose credit card or charge card,1
13837,Bank account or service Checking account,3


In [109]:
df_results.to_csv("complaints_with_topics.csv")